In [2]:
import requests
import json
import pandas as pd
from IPython.display import display
import math
from datetime import datetime
#from google.cloud import storage


In [3]:
apiURL = "https://hk.centanet.com/findproperty/api/Transaction/Search"
requestBody = {"postType": "Both",
               "day": "Day1095",
               "sort": "InsOrRegDate",
               "order": "Descending",
               "size": 48,
               "offset": 0,
               "pageSource": "search"
               }
requestHeaders = {
    "Lang": "en", "Content-Type": "application/json", "Accept": "gzip, deflate, br"}
response = requests.post(apiURL, data=json.dumps(
    requestBody), headers=requestHeaders)
df_raw = pd.DataFrame([response.json()])


In [8]:
df_transform = df_raw.explode('data')[['data']]
df_transform = df_transform['data'].apply(pd.Series)
df_transform2 = pd.concat([df_transform, df_transform['scope'].apply(
    pd.Series)], axis=1).drop('scope', axis=1)


In [4]:
def get_response_batch(url: str, response_size: int, offset: int) -> str:
    """Calls API and returns response json string."""

    request_body = {"postType": "Both",
                    "day": "Day1095",
                    "sort": "InsOrRegDate",
                    "order": "Descending",
                    "size": response_size,
                    "offset": "{0}".format(offset),
                    "pageSource": "search"
                    }
    request_headers = {"Lang": "en",
                       "Content-Type":
                       "application/json",
                       "Accept": "gzip, deflate, br"
                       }
    response = requests.post(url, data=json.dumps(
        request_body), headers=request_headers)
    return response

# Loop Count
var_loop_max_size = 0

# Initialize dataframe
df_clean = pd.DataFrame()

 # Initialize variables
 # "https://hk.centanet.com/findproperty/api/Transaction/Search"
api_url = "https://hk.centanet.com/findproperty/api/Transaction/Search"
var_api_response_size = 48

   # Get Loop value i.e. count of results/batch size
row_Count = get_response_batch(
api_url, var_api_response_size, 0).json().get("count", 0)

var_loop_max_size = math.ceil(row_Count/var_api_response_size)

    # Loop
var_loop = 0
while var_loop < var_loop_max_size:
    var_offset = var_loop * var_api_response_size

    df_raw = pd.DataFrame(
        [get_response_batch(api_url, var_api_response_size, var_offset).json()])

    df_transform = df_raw.explode('data')[['data']]
    df_transform = df_transform['data'].apply(pd.Series)
    df_clean = pd.concat([df_clean, df_transform])
    var_loop += 1

## WRITE TO GOOGLE CLOUD STORAGE project-diver-{env}##
# var_bucket_name = os.environ.get('DESTINATION_BUCKET_NAME')
# var_year = str(datetime.datetime.utcnow().strftime('%Y'))
# var_datetime = str(datetime.datetime.utcnow().strftime('%d%m%y'))
var_file_name = './DATASETS/centanet/historic/centanet_property_transaction.parquet'


In [10]:
df_clean.to_csv(
    './DATASETS/centanet/centanet_property_transaction_historic.csv')


In [ ]:

# Initialize function for executing API
def get_Response_Batch(par_URL, par_Response_Size, par_offset):
    request_Body = {"postType": "Both",
                    "day": "Day30",
                    "sort": "InsOrRegDate",
                    "order": "Descending",
                    "size": par_Response_Size,
                    "offset": "{0}".format(par_offset),
                    "pageSource": "search"
                    }
    request_Headers = {
        "Lang": "en", "Content-Type": "application/json", "Accept": "gzip, deflate, br"}
    response = requests.post(par_URL, data=json.dumps(
        request_Body), headers=request_Headers)
    return response


# Loop Count
var_Loop_max_size = 0

# Initialize dataframe
df_clean = pd.DataFrame()

# Initialize variables
api_URL = "https://hk.centanet.com/findproperty/api/Transaction/Search"
var_api_response_size = 48

# Get Loop value i.e. count of results/batch size
var_Loop_max_size = math.ceil(get_Response_Batch(
    api_URL, var_api_response_size, 0).json()['count']/var_api_response_size)

# Loop
var_loop = 0
while var_loop < var_Loop_max_size:
    var_offset = var_loop * var_api_response_size
    df_raw = pd.DataFrame(
        [get_Response_Batch(api_URL, var_api_response_size, var_offset).json()])
    df_transform = df_raw.explode('data')[['data']]
    df_transform = df_transform['data'].apply(pd.Series)
    df_clean = pd.concat([df_clean, df_transform])
    var_loop += 1

# WRITE TO GOOGLE CLOUD STORAGE project-diver-{env}
df_clean.to_csv('housePrice_{0}.csv'.format(str(datetime.utcnow())))


In [ ]:
# Initialize function for executing API
def get_Response_Batch(par_URL, par_Response_Size, par_offset):
    request_Body = {"postType": "Both",
                    "day": "Day30",
                    "sort": "InsOrRegDate",
                    "order": "Descending",
                    "size": par_Response_Size,
                    "offset": "{0}".format(par_offset),
                    "pageSource": "search"
                    }
    request_Headers = {
        "Lang": "en", "Content-Type": "application/json", "Accept": "gzip, deflate, br"}
    response = requests.post(par_URL, data=json.dumps(
        request_Body), headers=request_Headers)
    return response


def upload_blob(par_bucket_name, par_file_name, par_file):
    client_google_cloud = storage.Client()
    bucket = client_google_cloud.get_bucket(par_bucket_name)
    blob = bucket.blob(par_file_name)
    blob.upload_from_string(par_file)


# Loop Count
var_Loop_max_size = 0

# Initialize dataframe
df_clean = pd.DataFrame()

# Initialize variables
api_URL = "https://hk.centanet.com/findproperty/api/Transaction/Search"
var_api_response_size = 48


# Get Loop value i.e. count of results/batch size
var_Loop_max_size = math.ceil(get_Response_Batch(
    api_URL, var_api_response_size, 0).json()['count']/var_api_response_size)

# Loop
var_loop = 0
while var_loop < var_Loop_max_size:
    var_offset = var_loop * var_api_response_size
    df_raw = pd.DataFrame(
        [get_Response_Batch(api_URL, var_api_response_size, var_offset).json()])
    df_transform = df_raw.explode('data')[['data']]
    df_transform = df_transform['data'].apply(pd.Series)
    df_clean = pd.concat([df_clean, df_transform])
    var_loop += 1

# WRITE TO GOOGLE CLOUD STORAGE project-diver-{env}
var_bucket_name = 'project-diver-dev'
var_file_name = 'housePrice_{0}.csv'.format(str(datetime.utcnow()))
upload_blob(var_bucket_name, var_file_name, df_clean)
